<a href="https://colab.research.google.com/github/an-eve/nlp-nli-idioms/blob/main/Fine_Tuning_RoBERTa_on_IMPLI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning RoBERTa on IMPLI dataset and Exploring the dependence on the amount of data used

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
! pip install datasets
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 60.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [3]:
import torch
import numpy as np
import pandas as pd
import os
import copy
import datetime
import csv

from transformers import (RobertaTokenizer,
                          AutoModelForSequenceClassification,
                          Trainer,
                          TrainingArguments)
from datasets import (Dataset,
                      load_dataset,
                      concatenate_datasets,
                      load_metric,
                      ClassLabel,
                      Features)

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True, timeout_ms = 0)

Mounted at /content/drive


In [5]:
base_dir = '/content/drive/My Drive/'

In [6]:
BATCH_SIZE = 32

In [7]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda:0


## Model

In [8]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [9]:
def tokenize_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], padding="max_length", truncation=True)

In [10]:
model = AutoModelForSequenceClassification.from_pretrained("an-eve/roberta-base-mnli-2-labels")

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [11]:
metric = load_metric('glue', "mnli")
metric_name = "accuracy"

<ipython-input-11-64467277d4ee>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', "mnli")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [12]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

## Uploading datsets and Tokenization

In [13]:
test_data = load_dataset("an-eve/test_idioms", split='train')
tokenized_test_dataset = test_data.map(tokenize_function, batched=True)

Generating train split:   0%|          | 0/3350 [00:00<?, ? examples/s]

Map:   0%|          | 0/3350 [00:00<?, ? examples/s]

In [14]:
test_ne_data = load_dataset("an-eve/test_non_entailment_idioms", split='train')
tokenized_test_ne_dataset = test_ne_data.map(tokenize_function, batched=True)

Generating train split:   0%|          | 0/963 [00:00<?, ? examples/s]

Map:   0%|          | 0/963 [00:00<?, ? examples/s]

In [15]:
test_e_data = load_dataset("an-eve/test_entailment_idioms", split='train')
tokenized_test_e_dataset = test_e_data.map(tokenize_function, batched=True)

Generating train split:   0%|          | 0/2387 [00:00<?, ? examples/s]

Map:   0%|          | 0/2387 [00:00<?, ? examples/s]

In [16]:
num_shards = 9
train_sets = []

for i in range(num_shards):
     train_set= load_dataset(f"an-eve/train_sets_{i+1}_idioms", split='train')
     train_sets.append(train_set)

Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/4528 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/6791 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/9054 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/11317 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/13580 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/15843 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/18106 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20369 [00:00<?, ? examples/s]

In [17]:
tokenized_train_sets = []

for i in range(num_shards):
    tokenized_data = train_sets[i].map(tokenize_function, batched=True)
    tokenized_train_sets.append(tokenized_data)

Map:   0%|          | 0/2264 [00:00<?, ? examples/s]

Map:   0%|          | 0/4528 [00:00<?, ? examples/s]

Map:   0%|          | 0/6791 [00:00<?, ? examples/s]

Map:   0%|          | 0/9054 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


...

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Map:   0%|          | 0/18106 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Map:   0%|          | 0/20369 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [18]:
tokenized_train_sets[2]

Dataset({
    features: ['premise', 'hypothesis', 'label', 'input_ids', 'attention_mask'],
    num_rows: 6791
})

## Fine-tuning RoBERTa on IMPLI gradually increasing the amount of data

Evaluation on test set without fine-tuning

In [19]:
eval_folder = base_dir + "RoBERTa-test-idioms-full-" + str(datetime.datetime.now().timestamp())

if os.path.exists(eval_folder) == False:
    os.mkdir(eval_folder)


args_test = TrainingArguments(eval_folder,
         per_device_eval_batch_size=BATCH_SIZE)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_test_dataset,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics)

metrics = eval_trainer.evaluate()

eval_trainer.log_metrics("eval", metrics)
eval_trainer.save_metrics("eval", metrics)

***** eval metrics *****
  eval_accuracy           =     0.7033
  eval_loss               =     1.1458
  eval_runtime            = 0:00:25.69
  eval_samples_per_second =    130.374
  eval_steps_per_second   =      4.086


In [21]:
eval_folder = base_dir + "RoBERTa-test-idioms-ne-" + str(datetime.datetime.now().timestamp())

if os.path.exists(eval_folder) == False:
    os.mkdir(eval_folder)


args_test = TrainingArguments(eval_folder,
         per_device_eval_batch_size=BATCH_SIZE)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_test_ne_dataset,
    eval_dataset=tokenized_test_ne_dataset,
    compute_metrics=compute_metrics)

metrics = eval_trainer.evaluate()

eval_trainer.log_metrics("eval", metrics)
eval_trainer.save_metrics("eval", metrics)

***** eval metrics *****
  eval_accuracy           =     0.2305
  eval_loss               =     3.1833
  eval_runtime            = 0:00:07.16
  eval_samples_per_second =    134.462
  eval_steps_per_second   =      4.328


In [22]:
eval_folder = base_dir + "RoBERTa-test-idioms-e-" + str(datetime.datetime.now().timestamp())

if os.path.exists(eval_folder) == False:
    os.mkdir(eval_folder)


args_test = TrainingArguments(eval_folder,
         per_device_eval_batch_size=BATCH_SIZE)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_test_e_dataset,
    eval_dataset=tokenized_test_e_dataset,
    compute_metrics=compute_metrics)

metrics = eval_trainer.evaluate()

eval_trainer.log_metrics("eval", metrics)
eval_trainer.save_metrics("eval", metrics)

***** eval metrics *****
  eval_accuracy           =      0.894
  eval_loss               =     0.3238
  eval_runtime            = 0:00:17.66
  eval_samples_per_second =    135.142
  eval_steps_per_second   =      4.246


Evaluation on test set with incremental fine-tuning on train sets

In [23]:
model_mnli = copy.deepcopy(model)

In [24]:
# Training function for convenience

def train_eval_func(model, ind, type=None):

    folder = base_dir + f"RoBERTa-test-idioms-{type}-{ind+1}-" + str(datetime.datetime.now().timestamp())

    if os.path.exists(folder) == False:
        os.mkdir(folder)

    if type == "full":
        eval_set = tokenized_test_dataset
    elif type == "ne":
        eval_set = tokenized_test_ne_dataset
    else:
        eval_set = tokenized_test_e_dataset


    args = TrainingArguments(
    output_dir=folder,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    resume_from_checkpoint=True,
    push_to_hub=False)

    trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train_sets[ind],
    eval_dataset=eval_set,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)

    trainer.train()

    metrics = trainer.evaluate()

    trainer.log_metrics("eval", metrics)
    trainer.save_metrics("eval", metrics)

In [25]:
# Evaluation function for convenience

def eval_func(model, ind, type=None):

    folder = base_dir + f"RoBERTa-test-idioms-{type}-{ind+1}-" + str(datetime.datetime.now().timestamp())

    if os.path.exists(folder) == False:
        os.mkdir(folder)

    if type == "full":
        eval_set = tokenized_test_dataset
    elif type == "ne":
        eval_set = tokenized_test_ne_dataset
    else:
        eval_set = tokenized_test_e_dataset

    args_test = TrainingArguments(folder,
         per_device_eval_batch_size=BATCH_SIZE)

    eval_trainer = Trainer(
        model=model,
        args=args_test,
        train_dataset=eval_set,
        eval_dataset=eval_set,
        compute_metrics=compute_metrics)

    metrics = eval_trainer.evaluate()

    eval_trainer.log_metrics("eval", metrics)
    eval_trainer.save_metrics("eval", metrics)

In [26]:
num_shards = 9

for i in range(num_shards):

    model = copy.deepcopy(model_mnli)

    train_eval_func(model, i, type="full")
    eval_func(model, i, type="ne")
    eval_func(model, i, type="e")

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.567673,0.754030


***** eval metrics *****
  epoch                   =        1.0
  eval_accuracy           =      0.754
  eval_loss               =     0.5677
  eval_runtime            = 0:00:24.79
  eval_samples_per_second =    135.133
  eval_steps_per_second   =      4.236


***** eval metrics *****
  eval_accuracy           =     0.3281
  eval_loss               =     1.4013
  eval_runtime            = 0:00:07.12
  eval_samples_per_second =    135.218
  eval_steps_per_second   =      4.353


***** eval metrics *****
  eval_accuracy           =     0.9258
  eval_loss               =     0.2314
  eval_runtime            = 0:00:17.63
  eval_samples_per_second =     135.35
  eval_steps_per_second   =      4.253


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.553530,0.780896


...

***** eval metrics *****
  eval_accuracy           =     0.9321
  eval_loss               =     0.2051
  eval_runtime            = 0:00:17.60
  eval_samples_per_second =    135.555
  eval_steps_per_second   =      4.259


Epoch,Training Loss,Validation Loss,Accuracy
1,0.232000,0.531616,0.842388


***** eval metrics *****
  epoch                   =        1.0
  eval_accuracy           =     0.8424
  eval_loss               =     0.5316
  eval_runtime            = 0:00:24.80
  eval_samples_per_second =    135.069
  eval_steps_per_second   =      4.233


***** eval metrics *****
  eval_accuracy           =     0.6355
  eval_loss               =     1.2932
  eval_runtime            = 0:00:07.14
  eval_samples_per_second =    134.733
  eval_steps_per_second   =      4.337


***** eval metrics *****
  eval_accuracy           =     0.9258
  eval_loss               =     0.2244
  eval_runtime            = 0:00:17.66
  eval_samples_per_second =    135.112
  eval_steps_per_second   =      4.245


In [ ]:
torch.cuda.empty_cache()

## Evaluating on MNLI after fine-tuning on IMPLI